In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [11]:
df = pd.read_csv('Bengaluru_House_Data.csv')
df.head(10)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
6,Super built-up Area,18-May,Old Airport Road,4 BHK,Jaades,2732,4.0,NaN,204.00
7,Super built-up Area,Ready To Move,Rajaji Nagar,4 BHK,Brway G,3300,4.0,NaN,600.00
8,Super built-up Area,Ready To Move,Marathahalli,3 BHK,NaN,1310,3.0,1.0,63.25
9,Plot Area,Ready To Move,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.00


In [14]:
df.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

Let's essentially drop columns that are not used in Regression. Values that are not continuous

In [16]:
data= df.drop(['area_type','availability','society', 'balcony'], axis='columns')
data.head(10)

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00
5,Whitefield,2 BHK,1170,2.0,38.00
6,Old Airport Road,4 BHK,2732,4.0,204.00
7,Rajaji Nagar,4 BHK,3300,4.0,600.00
8,Marathahalli,3 BHK,1310,3.0,63.25
9,Gandhi Bazar,6 Bedroom,1020,6.0,370.00


Data cleaning process. To remove or edit the columns with null values. For example bath has 73 missing values, I have the option to either drop them, or fill them out with Measures of spread, ie Mean, Median etc. But when you check the shape, we have 13k rows. Dropping 73 will do no harm. So I dropped them. 

In [17]:
data.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [19]:
data = data.dropna()
data.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

The size columns seems to have a variety of values. Let me display the unique values it has

In [20]:
data['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [24]:
data['bedrooms'] = data['size'].apply(lambda x: int(x.split(' ')[0]))

In [25]:
data.head(10)

,location,size,total_sqft,bath,price,BHK,bedrooms
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3,3
4,Kothanur,2 BHK,1200,2.0,51.00,2,2
5,Whitefield,2 BHK,1170,2.0,38.00,2,2
6,Old Airport Road,4 BHK,2732,4.0,204.00,4,4
7,Rajaji Nagar,4 BHK,3300,4.0,600.00,4,4
8,Marathahalli,3 BHK,1310,3.0,63.25,3,3
9,Gandhi Bazar,6 Bedroom,1020,6.0,370.00,6,6


In [27]:
data['bedrooms'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

43 Bedrooms looked funny. I decided to check it out, to see whether the info matches. I prompted to return any data that has more than 20 bedrooms.

In [29]:
data[data['bedrooms'] > 20]

,location,size,total_sqft,bath,price,BHK,bedrooms
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43,43


I found the 27 bedroom have to have an area of 8000 square feet, while that with 43 having 2400. That to me seems odd. So readily, that is an outlier. It will really skew the results. I propose to drop it. Let me check the unique values in the total square feet column.

In [30]:
data['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

There are values that occur in ranges -> '1133 - 1384'. I want to check them out.

In [31]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [32]:
data[~data['total_sqft'].apply(is_float)]


,location,size,total_sqft,bath,price,BHK,bedrooms
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2,2
...,...,...,...,...,...,...,...
12975,Whitefield,2 BHK,850 - 1060,2.0,38.190,2,2
12990,Talaghattapura,3 BHK,1804 - 2273,3.0,122.000,3,3
13059,Harlur,2 BHK,1200 - 1470,2.0,72.760,2,2
13265,Hoodi,2 BHK,1133 - 1384,2.0,59.135,2,2


In [33]:
data[~data['total_sqft'].apply(is_float)].head(10)


,location,size,total_sqft,bath,price,BHK,bedrooms
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4,4


In [35]:
data[~data['total_sqft'].apply(is_float)]['total_sqft'].unique()


array(['2100 - 2850', '3067 - 8156', '1042 - 1105', '1145 - 1340',
       '1015 - 1540', '34.46Sq. Meter', '1195 - 1440', '4125Perch',
       '1120 - 1145', '3090 - 5002', '1160 - 1195', '1000Sq. Meter',
       '1115 - 1130', '1100Sq. Yards', '520 - 645', '1000 - 1285',
       '650 - 665', '633 - 666', '5.31Acres', '30Acres', '1445 - 1455',
       '884 - 1116', '850 - 1093', '716Sq. Meter', '547.34 - 827.31',
       '580 - 650', '3425 - 3435', '1804 - 2273', '3630 - 3800',
       '4000 - 5249', '1500Sq. Meter', '142.61Sq. Meter', '1574Sq. Yards',
       '1250 - 1305', '670 - 980', '1005.03 - 1252.49', '1004 - 1204',
       '361.33Sq. Yards', '645 - 936', '2710 - 3360', '2830 - 2882',
       '596 - 804', '1255 - 1863', '1300 - 1405', '117Sq. Yards',
       '934 - 1437', '980 - 1030', '2249.81 - 4112.19', '1070 - 1315',
       '3040Sq. Meter', '500Sq. Yards', '2806 - 3019', '613 - 648',
       '704 - 730', '1210 - 1477', '3369 - 3464', '1125 - 1500',
       '167Sq. Meter', '1076 - 1199',

In [36]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [37]:
data2 = data.copy()


In [38]:
data2['total_sqft'] = data2['total_sqft'].apply(convert_sqft_to_num)
data2.head(10)

,location,size,total_sqft,bath,price,BHK,bedrooms
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,2
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,2
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,4
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,4
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,3
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6,6


In [39]:
data2.loc[410]

location      Kengeri
size            1 BHK
total_sqft        NaN
bath              1.0
price            18.5
BHK                 1
bedrooms            1
Name: 410, dtype: object

In [40]:
data2.head()

,location,size,total_sqft,bath,price,BHK,bedrooms
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,2


Feature Engineering. 
I want to introduce a column that shall be used as a metric, where we calculate the cost in terms of each square_ft. 
A calculated column 

In [41]:
data2['price_per_sqft'] = ( data2['price'] * 100000 ) / data2['total_sqft']
data2.head()

,location,size,total_sqft,bath,price,BHK,bedrooms,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,2,4250.000000


In [43]:
data2.shape

(13246, 8)

In [45]:
len(data2['location'].unique())

1304

There are 1304 unique locations in this data. We can try reduce them . Let's first find how these locations are distributed in terms of count. And arrange the values in descending order .

In [47]:
data2.groupby('location')['location'].agg('count').sort_values(ascending=False)

location
Whitefield             534
Sarjapur  Road         392
Electronic City        302
Kanakpura Road         266
Thanisandra            233
                      ... 
 Banaswadi               1
Kanakadasa Layout        1
Kanakapur main road      1
Kanakapura  Rod          1
whitefiled               1
Name: location, Length: 1304, dtype: int64

I want to group all the locations that have values less than 10 into one group. Let's say Other. Let's find out how many there are

In [48]:
location_count = data2.groupby('location')['location'].agg('count')
location_count 

location
 Anekal                 1
 Banaswadi              1
 Basavangudi            1
 Bhoganhalli            1
 Devarabeesana Halli    6
                       ..
t.c palya               1
tc.palya                4
vinayakanagar           1
white field,kadugodi    1
whitefiled              1
Name: location, Length: 1304, dtype: int64

In [50]:
len(location_count[location_count <= 10])

1063

In [52]:
locations_less_than_10 = location_count[location_count <= 10]
locations_less_than_10

location
 Anekal                 1
 Banaswadi              1
 Basavangudi            1
 Bhoganhalli            1
 Devarabeesana Halli    6
                       ..
t.c palya               1
tc.palya                4
vinayakanagar           1
white field,kadugodi    1
whitefiled              1
Name: location, Length: 1063, dtype: int64

I am going to change the location value in the dataframe, such that, if it has less than 10 houses, it is grouped as other.

In [61]:
data2['location'] = data2['location'].apply(lambda x: 'Other' if x in locations_less_than_10 else x)

In [62]:
len(data2['location'].unique())

242

In [63]:
data2.head(20)

,location,size,total_sqft,bath,price,BHK,bedrooms,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,2,4250.000000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,2,3247.863248
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,4,7467.057101
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,4,18181.818182
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,6,36274.509804
